In [1]:
!pip install keras_vggface keras_applications mtcnn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 55.4 MB/s eta 0:00:00


In [2]:
import os
img_folder_path = 'drive/MyDrive/FGNET/images'
os.listdir('drive/MyDrive/FGNET/images')

['006',
 '001',
 '010',
 '002',
 '004',
 '008',
 '007',
 '009',
 '003',
 '005',
 '015',
 '017',
 '012',
 '011',
 '016',
 '018',
 '013',
 '020',
 '019',
 '014',
 '024',
 '030',
 '028',
 '029',
 '027',
 '022',
 '021',
 '025',
 '026',
 '023',
 '035',
 '031',
 '038',
 '036',
 '037',
 '040',
 '033',
 '034',
 '032',
 '039',
 '042',
 '046',
 '045',
 '041',
 '047',
 '043',
 '050',
 '044',
 '049',
 '048',
 '058',
 '055',
 '059',
 '060',
 '051',
 '056',
 '054',
 '053',
 '052',
 '057',
 '062',
 '069',
 '068',
 '070',
 '066',
 '063',
 '064',
 '067',
 '065',
 '061',
 '078',
 '075',
 '071',
 '077',
 '080',
 '076',
 '072',
 '074',
 '079',
 '073',
 '082',
 '081']

In [3]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from numpy import asarray
import sys

# Build model

In [6]:
import os
import pandas as pd
import numpy as np
import tensorflow.keras as keras
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam

In [12]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)

train_generator = \
    train_datagen.flow_from_directory(
img_folder_path,
target_size=(224,224),
color_mode='rgb',
batch_size=32,
class_mode='categorical',
shuffle=True)

Found 1002 images belonging to 82 classes.


In [8]:
train_generator.class_indices.values()
# dict_values([0, 1, 2])
NO_CLASSES = len(train_generator.class_indices.values())
NO_CLASSES

82

In [11]:
filename = "/usr/local/lib/python3.9/dist-packages/keras_vggface/models.py"
text = open(filename).read()
open(filename, "w+").write(text.replace('keras.engine.topology', 'tensorflow.keras.utils'))

from keras_vggface.vggface import VGGFace

base_model = VGGFace(include_top=True,
    model='vgg16',
    input_shape=(224, 224, 3))
base_model.summary()

print(len(base_model.layers))
# 26 layers in the original VGG-Face

580070376/580070376 [==============================] - 20s 0us/step
Model: "vggface_vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1_1 (Conv2D)            (None, 224, 224, 64)      1792      
                                                                 
 conv1_2 (Conv2D)            (None, 224, 224, 64)      36928     
                                                                 
 pool1 (MaxPooling2D)        (None, 112, 112, 64)      0         
                                                                 
 conv2_1 (Conv2D)            (None, 112, 112, 128)     73856     
                                                                 
 conv2_2 (Conv2D)            (None, 112, 112, 128)     147584    
                                                   

In [ ]:
base_model = VGGFace(include_top=False,
model='vgg16',
input_shape=(224, 224, 3))
base_model.summary()
print(len(base_model.layers))
# 19 layers after excluding the last few layers

58909280/58909280 [==============================] - 2s 0us/step
Model: "vggface_vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1_1 (Conv2D)            (None, 224, 224, 64)      1792      
                                                                 
 conv1_2 (Conv2D)            (None, 224, 224, 64)      36928     
                                                                 
 pool1 (MaxPooling2D)        (None, 112, 112, 64)      0         
                                                                 
 conv2_1 (Conv2D)            (None, 112, 112, 128)     73856     
                                                                 
 conv2_2 (Conv2D)            (None, 112, 112, 128)     147584    
                                                      

In [14]:
x = base_model.output
x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)

# final layer with softmax activation
x = Dense(NO_CLASSES, activation='softmax')(x)
transfer_model = Model(inputs = base_model.input, outputs = x)

In [15]:
print(transfer_model.summary())
print(len(transfer_model.layers))


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1_1 (Conv2D)            (None, 224, 224, 64)      1792      
                                                                 
 conv1_2 (Conv2D)            (None, 224, 224, 64)      36928     
                                                                 
 pool1 (MaxPooling2D)        (None, 112, 112, 64)      0         
                                                                 
 conv2_1 (Conv2D)            (None, 112, 112, 128)     73856     
                                                                 
 conv2_2 (Conv2D)            (None, 112, 112, 128)     147584    
                                                                 
 pool2 (MaxPooling2D)        (None, 56, 56, 128)       0   

In [16]:
# don't train the first 19 layers - 0..18
for layer in transfer_model.layers[:19]:
    layer.trainable = False

# train the rest of the layers - 19 onwards
for layer in transfer_model.layers[19:]:
    layer.trainable = True


In [24]:
transfer_model.compile(optimizer='Adam',
    loss='categorical_crossentropy')


In [25]:
transfer_model.fit(train_generator,
  batch_size = 1,
  verbose = 1,
  epochs = 100)

Epoch 1/100
32/32 [==============================] - 32s 921ms/step - loss: 4.3874
Epoch 2/100
32/32 [==============================] - 7s 204ms/step - loss: 4.3848
Epoch 3/100
32/32 [==============================] - 6s 195ms/step - loss: 4.3851
Epoch 4/100
32/32 [==============================] - 7s 226ms/step - loss: 4.3854
Epoch 5/100
32/32 [==============================] - 7s 215ms/step - loss: 4.3849
Epoch 6/100
32/32 [==============================] - 7s 203ms/step - loss: 4.3849
Epoch 7/100
32/32 [==============================] - 6s 187ms/step - loss: 4.3849
Epoch 8/100
32/32 [==============================] - 7s 214ms/step - loss: 4.3848
Epoch 9/100
32/32 [==============================] - 7s 204ms/step - loss: 4.3847
Epoch 10/100
32/32 [==============================] - 7s 213ms/step - loss: 4.3845
Epoch 11/100
32/32 [==============================] - 6s 185ms/step - loss: 4.3847
Epoch 12/100
32/32 [==============================] - 7s 216ms/step - loss: 4.3844
Epoch 13/100

In [27]:
# creates a HDF5 file
transfer_model.save('drive/MyDrive/missing child model/vgg_transfer_learning_model.h5')